In [1]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 17.4 MB/s 
     |████████████████████████████████| 578.0 MB 17 kB/s 
     |████████████████████████████████| 438 kB 62.2 MB/s 
     |████████████████████████████████| 5.9 MB 53.4 MB/s 
     |████████████████████████████████| 1.7 MB 54.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [3]:
import numpy as np
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

train = pd.read_csv("/content/drive/MyDrive/2018-2023 WPI/2022-2023 WPI Master's/Fall Semester/Machine Learning/CS 539 Final Project/Data Split/train_preprocessing.csv")
validate = pd.read_csv("/content/drive/MyDrive/2018-2023 WPI/2022-2023 WPI Master's/Fall Semester/Machine Learning/CS 539 Final Project/Data Split/valid_preprocessing.csv")
test = pd.read_csv("/content/drive/MyDrive/2018-2023 WPI/2022-2023 WPI Master's/Fall Semester/Machine Learning/CS 539 Final Project/Data Split/test_preprocessing.csv")

Mounted at /content/drive/


In [5]:
train_x = train['text']
train_y = train['target']
validate_x = validate['text']
validate_y = validate['target']
test_x = test['text']
test_y = test['target']

In [6]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [7]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']


In [10]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras import regularizers

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_x)
sequences = tokenizer.texts_to_sequences(train_x)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

AttributeError: ignored

In [9]:
tweets

NameError: ignored

In [22]:
#get_sentence_embeding(train_x[0])

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [25]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [26]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [27]:
model.fit(train_x, train_y, epochs=10)

ValueError: ignored